In [1]:
#load CSVs as lists of lists of strings
#Later on the training set is split up to make a validation set, so we're shuffling it here before any further processing

import csv
import numpy


trainraw = []
testraw = []

with open('./Titanic/train.csv', 'rb') as traincsv:
    trainreader = csv.reader(traincsv, delimiter=',', quotechar='|')
    for row in trainreader:
        #print ', '.join(row)
        #print row
        trainraw.append(row)
        
with open('./Titanic/test.csv', 'rb') as testcsv:
    testreader = csv.reader(testcsv, delimiter=',', quotechar='|')
    for row in testreader:
        #print ', '.join(row)
        #print row
        testraw.append(row)

In [2]:
"""
Age field has missing data values; this calculates the average and uses that as a replacement
I tried culling the data but it didn't really make a big difference on NNs, 
but might make a difference in some of the other algorithms I wanted to try.
The kaggle forums suggest extrapolating age from names and the other data, I'd probably try that next
"""
trainAgeAvg=-1
testAgeAvg=-1
trainAges=[]
testAges=[]

for x in range(1, len(trainraw)):
    try:
        trainAges.append(int(trainraw[x][6]))
    except ValueError:
        pass
for x in range(1, len(testraw)):
    try:
        testAges.append(int(testraw[x][5]))
    except ValueError:
        pass
    
trainAgeAvg=numpy.mean(trainAges)
testAgeAvg=numpy.mean(testAges)

#print trainAgeAvg
#print testAgeAvg

In [3]:
"""
Data cleanup for training data, changing things to float, with one-hot encoding and dummy variables as needed.
Cabin and ticket data are discarded due to so many missing fields. Assuming blanks in the fare field meant free passage.
"""

traindat=[]
for x in range(1, len(trainraw)):
    temp=[]
       
    if(trainraw[x][2]=='1'):
        temp.append(1.0)
        temp.append(0.0)
        temp.append(0.0)
    if(trainraw[x][2]=='2'):
        temp.append(0.0)
        temp.append(1.0)
        temp.append(0.0)
    if(trainraw[x][2]=='3'):
        temp.append(0.0)
        temp.append(0.0)
        temp.append(1.0)
    
    if(trainraw[x][5]=="male"):
        temp.append(0.0)
    elif(trainraw[x][5]=="female"):
        temp.append(1.0)
    else:
        temp.append(2)
        print ("shit happened")
        
    try:
        temp.append(float(trainraw[x][6])) #age
    except ValueError:
        temp.append(trainAgeAvg)
       
    temp.append(float(trainraw[x][7])) #Siblings/Spouse?
    
    temp.append(float(trainraw[x][8])) #Parent/Child
    
    try:
        temp.append(float(trainraw[x][10])) #fare
    except ValueError:
        temp.append(0)
        
    if(trainraw[x][12]=='S'):
        temp.append(1.0)
        temp.append(0.0)
        temp.append(0.0)
    if(trainraw[x][12]=='C'):
        temp.append(0.0)
        temp.append(1.0)
        temp.append(0.0)
    if(trainraw[x][12]=='Q'):
        temp.append(0.0)
        temp.append(0.0)
        temp.append(1.0)
    
    #print temp
    #temp2=[]
    #temp2.append(temp)
    #temp2.append(trainraw[x][1])
    #temp2.append(trainraw[x][0])
    #traindat.append(temp2)
    traindat.append((temp))

In [4]:
"""
Data cleanup for test data, changing things to float, with one-hot encoding and dummy variables as needed
Cabin and ticket data are discarded due to so many missing fields. Assuming blanks in the fare field meant free passage.
"""

testdat=[]
for x in range(1, len(testraw)):
    temp=[]
    
    if(testraw[x][1]=='1'):
        temp.append(1)
        temp.append(0)
        temp.append(0)
    if(testraw[x][1]=='2'):
        temp.append(0)
        temp.append(1)
        temp.append(0)
    if(testraw[x][1]=='3'):
        temp.append(0)
        temp.append(0)
        temp.append(1)
    
    if(testraw[x][4]=="male"):
        temp.append(0)
    elif(testraw[x][4]=="female"):
        temp.append(1)
    else:
        temp.append(2)
        print ("shit happened")
        
    try:
        temp.append(float(testraw[x][5])) #age
    except ValueError:
        temp.append(testAgeAvg)
    
    temp.append(float(testraw[x][6])) #Siblings/Spouse?
    
    temp.append(float(testraw[x][7])) #Parent/Child
    
    try:
        temp.append(float(testraw[x][9])) #fare
    except ValueError:
        temp.append(0)
            
    if(testraw[x][11]=='S'):
        temp.append(1)
        temp.append(0)
        temp.append(0)
    if(testraw[x][11]=='C'):
        temp.append(0)
        temp.append(1)
        temp.append(0)
    if(testraw[x][11]=='Q'):
        temp.append(0)
        temp.append(0)
        temp.append(1)
    
    #print temp
    #temp2=[]
    #temp2.append(temp)
    #temp2.append(testraw[x][0])
    #testdat.append(temp2)
    testdat.append(temp)

In [5]:
"""
Most ML algorithms work best with feature scaling such that one feature does not overpower the rest.
I'm a little worried that the one-hot encoding may cause problems, but there doesn't seem to be much of a problem
"""

def normalize(data, col):
    coldat = []
    for x in range(1, len(data)):
        coldat.append(float(data[x][col]))
    max=numpy.nanmax(coldat)
    min=numpy.nanmin(coldat)
    scalefactor=float(max-min)
    print max
    print min
    #print scalefactor
    #print ''
    for x in range(1, len(data)):
        z=float(data[x][col])
        z=(z-min)/scalefactor
        data[x][col]=z


In [6]:
#normalize training data
normalize(traindat, 4)
normalize(traindat, 5)
normalize(traindat, 6)
normalize(traindat, 7)
#for x in range(1,10):
#    print traindat[x]

80.0
0.42
8.0
0.0
6.0
0.0
512.3292
0.0


In [7]:
#normalize test data
normalize(testdat, 4)
normalize(testdat, 5)
normalize(testdat, 6)
normalize(testdat, 7)
#for x in range(1,10):
#    print testdat[x]

76.0
0.17
8.0
0.0
9.0
0.0
512.3292
0.0


In [8]:
#converts training data to numpy floats so that it won't be an issue in later conversions
for x in range(0,len(traindat)):
    for y in range(0,len(traindat[x])):
        traindat[x][y]=numpy.float32(traindat[x][y])    

In [9]:
#converts testing data to numpy floats so that it won't be an issue in later conversions
for x in range(0,len(testdat)):
    for y in range(0,len(testdat[x])):
        testdat[x][y]=numpy.float32(testdat[x][y])

In [10]:
#Copying all the values to a new ndarray fixed some shape issues during training.
#Also splitting off 91 examples to use as a validation set. The data is shuffled before doing so.
#numpy.random.shuffle(traindat)

traindatfixed=numpy.zeros((891,11))
#trainval=numpy.zeros((91,11))

In [11]:
#copy training feature data
for x in range(0,len((traindat))):
    for y in range(0,len(traindat[x])):
        traindatfixed[x][y]=numpy.float32(traindat[x][y])             

In [12]:
#copy and one-hot training IV for softmax regression
survival = []
#survivalval=[]
for x in range(0,len(traindat)):
    if(float(trainraw[x+1][1])==0):
        survival.append(numpy.array([1.0,0.0]))
    else:
        survival.append(numpy.array([0.0,1.0]))

survival = numpy.array(survival)
#survivalval = numpy.array(survivalval)
    


In [13]:
#convert test data
testdatfixed=numpy.zeros((418,11))
for x in range(0,len(testdat)):
    for y in range(0,len(testdat[x])):
        testdatfixed[x][y]=numpy.float32(testdat[x][y])

In [1]:
#Sanity test with scikit-learn random forest, with code from the kaggle tutorial.
#Disabled because it loads a deprecated version of BLAS that theano complains about.
#Code to convert output to CSV needs tweaks to work;
#this outputs [0, 1] or [1, 0], as opposed to [0] or [1] from the NNs.

"""
from sklearn.ensemble import RandomForestClassifier 

forest = RandomForestClassifier(n_estimators = 100)

forest = forest.fit(traindatfixed,survival)

# Since I reserved some validation data, might as well use it
print (forest.score(traindatfixed,survival))

outs = forest.predict(testdatfixed)
"""

'\nfrom sklearn.ensemble import RandomForestClassifier \n\nforest = RandomForestClassifier(n_estimators = 100)\n\nforest = forest.fit(traindatfixed,survival)\n\n# Since I reserved some validation data, might as well use it\nprint (forest.score(traindatfixed,survival))\n\nouts = forest.predict(testdatfixed)\n'

In [15]:
"""
Runs a deep neural network on the dataset, uses code from the keras examples. 
Probably overkill, but this is an exercise anyway.
This section contains the imports.
"""
from __future__ import print_function
import numpy as np
np.random.seed(1337)  # for reproducibility

from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils

Using Theano backend.
Using gpu device 0: GeForce GTX 960 (CNMeM is enabled with initial size: 70.0% of memory, cuDNN 4007)


In [34]:
#NN adapted from the Keras examples
#I tried an autoencoder in front and hidden layers with <11 nodes. Appears that dimensionality reduction isn't to helpful
#A shallow NN with 7 hidden works surprisingly well, as it stands I need to do more feature engineering
#Dropout doesn't seem to help all that much, but there's no validation data
#I tried splitting off 91 random examples for validation but that hit accuracy quite hard


batch_size = 20
nb_classes = 2
nb_epochs = 50
models=[]

for x in range(0,10):
    nb_epoch=nb_epochs*x
    print(nb_epoch)
    model = Sequential()
    model.add(Dense(100, input_shape=(11,)))
    model.add(Activation('relu'))
    model.add(Dense(100))
    model.add(Activation('relu'))
    model.add(Dense(2))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

#Verbosity disabled since it triggers a juptyer notebook bug and crashes the training

    history = model.fit(traindatfixed, survival,
                    batch_size=batch_size, nb_epoch=nb_epoch,
                    verbose=0, validation_data=(traindatfixed, survival))

    score = model.evaluate(traindatfixed, survival, verbose=1)
    models.append(model)
    print('Test score:', score[0])
    print('Test accuracy:', score[1])
    print('')
print('\nDone')

0
891/891 [==============================] - 0s     
Test score: 0.708169247499
Test accuracy: 0.373737373804

50
891/891 [==============================] - 0s     
Test score: 0.368352231621
Test accuracy: 0.848484848017

100
891/891 [==============================] - 0s     
Test score: 0.349823465853
Test accuracy: 0.851851852521

150
891/891 [==============================] - 0s     
Test score: 0.325372112302
Test accuracy: 0.85970819371

200
891/891 [==============================] - 0s     
Test score: 0.316319748592
Test accuracy: 0.865319864852

250
891/891 [==============================] - 0s     
Test score: 0.307497302424
Test accuracy: 0.874298540497

300
891/891 [==============================] - 0s     
Test score: 0.323957799444
Test accuracy: 0.866442200444

350
891/891 [==============================] - 0s     
Test score: 0.289115972909
Test accuracy: 0.874298541634

400
891/891 [==============================] - 0s     
Test score: 0.277658376228
Test accuracy: 0.8

In [35]:
#Converts things to CSV for Kaggle submission

#model=models[0]
#outs = model.predict_classes(traindatfixed)
#print(outs)
outs = model.predict_classes(testdatfixed)
#print(outs)

output = []
output.append(["PassengerId","Survived"])

for x in range(0,len(outs)):
    output.append([testraw[x+1][0],int(outs[x])])
    
with open("outputDNN.csv", 'wb') as f:
    writer=csv.writer(f)
    writer.writerows(output)

891/891 [==============================] - 0s     
[0 1 0 1 0 0 0 0 1 1 1 1 0 0 0 1 0 0 0 1 0 0 1 0 0 0 0 0 1 0 0 1 1 0 1 0 0
 0 0 0 0 1 0 1 1 0 0 1 0 0 0 0 1 1 0 0 1 0 1 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0
 1 0 0 0 1 0 0 0 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 0 0 0 1 0
 0 0 0 1 0 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0 1 0 0 0 0 0 0 0
 0 0 0 1 0 0 0 0 1 0 0 0 0 1 0 0 0 1 1 0 1 1 0 0 1 0 0 0 0 0 0 0 0 0 0 1 1
 0 1 0 0 0 1 0 0 1 1 1 0 0 1 1 0 0 0 0 0 1 0 0 1 0 0 1 0 0 0 1 0 0 1 0 0 0
 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 1 0 0 0 1 0 0 0 0 0 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 0 0 0 1 0 0 0 1 1 1 0 1 0 1 1 0 0 0 0 0 0 0 0 1 0 0 0 0 1 1 1 0 0 0 0
 0 0 1 1 1 1 0 1 0 1 1 1 0 1 1 1 0 0 0 0 1 0 1 1 0 0 1 1 0 1 0 1 0 1 1 0 0
 0 1 0 0 1 0 0 1 1 0 0 0 1 1 0 1 0 0 0 0 0 0 0 1 1 1 1 0 0 0 0 0 0 1 1 1 1
 1 0 0 0 0 1 0 0 0 0 1 1 0 1 0 0 0 1 0 1 1 0 0 1 1 0 0 0 0 1 0 0 0 0 0 0 0
 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 0 1
 0 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 0 